# 2D UMAPs

Loading the R packages.

In [1]:
suppressWarnings({suppressPackageStartupMessages({
    library(Seurat)
    library(ggplot2)
    library(ggsci)
    library(readxl)
})})

Loading the scRNA-seq pre-processed and Leiden-clustered data.

In [2]:
hgsoc <- readRDS("HGSOC_CellHashing_CLUSTERED.RDS")

Loading mechanisms of action or high-level annotations' metadata.

In [3]:
high_anno <- as.data.frame(read_xlsx(path = "mechanisms_of_actions.xlsx", sheet = 1))
high_anno <- high_anno[, c(1:3)]
rownames(high_anno) <- high_anno$`Preferred name`

New names:
• `` -> `...4`
• `` -> `...5`


In [4]:
head(high_anno)

,Preferred name,Mechanism/Targets,Higher level classification
,<chr>,<chr>,<chr>
DMSO,DMSO,0,Control
Belinostat,Belinostat,HDAC inhibitor,HDAC inhibitor
Quisinostat,Quisinostat,HDAC inhibitor,HDAC inhibitor
Dinaciclib,Dinaciclib,CDK inhibitor,CDK inhibitor
Pictilisib,Pictilisib,"PI3K inhibitor, pan-class I",PI3K/mTOR/AKT inhibitor
Pacritinib,Pacritinib,FLT3/JAK2,Multi-kinase inhibitor


Associating each cell to the higher level annotation of the drug used to perturb them.

In [5]:
hgsoc@meta.data$higher_mechanism_class <- sapply(hgsoc@meta.data$Treatment_group, 
                                                 function(x) high_anno[x, "Higher level classification"])

Loading our custom palette generated with https://mokole.com/palette.html. This tool starts with a set of X11 colors (https://www.w3.org/TR/css3-iccprof#x11-color) and filters out the darkest and brightest colors based on specified minimum and maximum luminosity values. In our case, the chosen parameters were 20% for the minimum luminosity and 80% for the maximum luminosity. Then, it creates a subset of colors where each color is visually distinct from every other color by manipulating them in the CIE Lab color space, which accounts for perceptual differences (see https://www.datacolor.com/business-solutions/blog/what-is-cielab/). The resulting palettes are ranked based on the minimal pairwise distance between colors, with higher values indicating better distinctiveness. 

In [6]:
custom_palette <- c("Bcl-2 inhibitor" = "#2f4f4f", 
                    "BET inhibitor" = "#a52a2a", 
                    "Control" = "#bcbec0",
                    "CDK inhibitor" = "#228b22",
                    "CHK1 inhibitor" = "#4b0082",
                    "HDAC inhibitor" = "#ff8c00",
                    "IAPs/SMAC mimetic" = "#d2b48c",
                    "Multi-kinase inhibitor" = "#00ff00",
                    "PAK inhibitor" = "#00bfff",
                    "PARP inhibitor" = "#0000ff",
                    "PI3K/mTOR/AKT inhibitor" = "#ff1493",
                    "PLK1 inhibitor" = "#ffff54",
                    "Ras/Raf/MEK/ERK inhibitor" = "#dda0dd",
                    "XPO1/CRM1 inhibitor" = "#7fffd4")

Plotting the UMAPs:

In [7]:
Idents(hgsoc) <- "higher_mechanism_class"
levels(hgsoc) <- c("Control", sort(levels(hgsoc))[-which(sort(levels(hgsoc)) == "Control")])
ggsave(filename = "HGSOC_CellHashing_UMAP_MoAs.pdf", width = 9, height = 7, 
       plot = DimPlot(hgsoc, pt.size = 1, cols = custom_palette) + NoAxes())

Idents(hgsoc) <- "model"
levels(hgsoc) <- c("JHOS2", "PDC2", "PDC3") # just for the color palette, that we prefer being swapped.
ggsave(filename = "HGSOC_CellHashing_UMAP_Models.pdf", width = 8, height = 7, 
       plot = DimPlot(hgsoc, pt.size = 1, cols = pal_npg()(3)) + NoAxes())

Idents(hgsoc) <- "RNA_clusters"
ggsave(filename = "HGSOC_CellHashing_UMAP_RNA_clusters.pdf", width = 7.5, height = 7, 
DimPlot(hgsoc, pt.size = 1, cols = c(pal_npg()(10), pal_nejm()(3))) + NoAxes())

Idents(hgsoc) <- "Phase"
ggsave(filename = "HGSOC_CellHashing_UMAP_CellCycle.pdf", width = 7.5, height = 7, 
DimPlot(hgsoc, pt.size = 1, cols = pal_nejm()(3)) + NoAxes())

When saving them for publication purposes, the legend is removed so as to have the UMAPs of the same size. The legend is added back from the previously saved PDFs with Adobe Illustrator.

In [8]:
Idents(hgsoc) <- "higher_mechanism_class"
ggsave(filename = "HGSOC_CellHashing_UMAP_MoAs_noLegend.pdf", width = 7, height = 7, 
       plot = DimPlot(hgsoc, pt.size = 1, cols = custom_palette) + NoAxes() + NoLegend())

Idents(hgsoc) <- "model"
levels(hgsoc) <- c("JHOS2", "PDC2", "PDC3") # just for the color palette, that we prefer being swapped.
ggsave(filename = "HGSOC_CellHashing_UMAP_Models_noLegend.pdf", width = 7, height = 7, 
       plot = DimPlot(hgsoc, pt.size = 1, cols = pal_npg()(3)) + NoAxes() + NoLegend())

Idents(hgsoc) <- "RNA_clusters"
ggsave(filename = "HGSOC_CellHashing_UMAP_RNA_clusters_noLegend.pdf", width = 7, height = 7, 
DimPlot(hgsoc, pt.size = 1, cols = c(pal_npg()(10), pal_nejm()(3))) + NoAxes() + NoLegend())

Idents(hgsoc) <- "Phase"
ggsave(filename = "HGSOC_CellHashing_UMAP_CellCycle_noLegend.pdf", width = 7, height = 7, 
DimPlot(hgsoc, pt.size = 1, cols = pal_nejm()(3)) + NoAxes() + NoLegend())

In [9]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.8 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_analyses/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] readxl_1.4.1       ggsci_3.0.0        ggplot2_3.4.2      SeuratObject_4.1.3
[5] Seurat_4.3.0.9002 

loaded via a namespace (and not attached):
  [1] Rtsne_0.16             colorspace_2.1-0       deldir_1.0-6          
  [4] ellipsis_0.3.2         ggridges_0.5.4         